In [ ]:
import sire as sr

# Replica Exchange

The ease with which multiple simulations can be handled simultaneously allows for a simple implementation of replica exchange.

Load an example perturbable system

In [ ]:
mols = sr.load_test_files("merged_molecule.s3")

Create two replicas of the system, at two different lambda values

In [ ]:
rep0 = mols.dynamics(timestep="4fs", temperature="25oC", lambda_value=0.0)

In [ ]:
rep1 = mols.dynamics(timestep="4fs", temperature="25oC", lambda_value=0.2)

### Implementation of a minimal `replica_exchange` function

This function takes in a pair of sire `dynamics` objects and performs a Hamiltonian replica exchange move, returning the two systems as well as a boolean that indicates whether or not the move was accepted

In [ ]:
def replica_exchange(replica0, replica1):

    #Retrieve the information we need for each replica from the dynamics objects
    lam0 = replica0.get_lambda()
    lam1 = replica1.get_lambda()

    ensemble0 = replica0.ensemble()
    ensemble1 = replica1.ensemble()

    temperature0 = ensemble0.temperature()
    temperature1 = ensemble1.temperature()

    #The lambda_values argument allows us to retrieve the potential energy from both objects at both lambda values
    nrgs0 = replica0.current_potential_energy(lambda_values=[lam0,lam1])
    nrgs1 = replica1.current_potential_energy(lambda_values=[lam0,lam1])

    from sire.units import k_boltz

    beta0 = 1.0 / (k_boltz * temperature0)
    beta1 = 1.0 / (k_boltz * temperature1)

    #Check properties of the ensemble to see if we need to include a pressure term
    if not ensemble0.is_constant_pressure():
        delta = beta1 * (nrgs1[0] - nrgs1[1]) + beta0 * (nrgs0[0] - nrgs0[1])
    else:
        volume0 = replica0.current_space().volume()
        volume1 = replica1.current_space().volume()

        pressure0 = ensemble0.pressure()
        pressure1 = ensemble1.pressure()

        delta = beta1 * (
            nrgs1[0] - nrgs1[1] + pressure1 * (volume1 - volume0)
        ) + beta0 * (nrgs0[0] - nrgs0[1] + pressure0 * (volume0 - volume1))
    
    from math import exp
    import random

    move_passed = delta > 0 or (exp(delta) >= random.random())

    if move_passed:
        if lam0 != lam1:
            replica0.set_lambda(lam1)
            replica1.set_lambda(lam0)
        return (replica1, replica0, True)

    else:
        return (replica0, replica1, False)


Run dynamics on both replicas. We'll minimise each replica first, to prevent NaN errors. The error catching will mostly catch these and auto-minimise if found (i.e. you could comment out the minimisation lines)

In [ ]:
rep0.minimise()
rep0.run("5ps")

In [ ]:
rep1.minimise()
rep1.run("5ps")

Perform a replica exchange move between these two replicas. If the move passes, then the replicas are swapped (by swapping their lambda values). They are returned from this function in the same lambda order as they went in (i.e. in increasing lambda order)

In [ ]:
(rep0, rep1, swapped) = replica_exchange(rep0, rep1)

Was the move successful?

In [ ]:
print("Swapped?", swapped)

Even if they were swapped, the order of lambda is preserved

In [ ]:
print(rep0.get_lambda(), rep1.get_lambda())

#### This functionality also exists within the current version of sire (the sire version also supports temperature-based repex) and can be accessed with `sire.morph.replica_exchange`

In [ ]:
(rep0, rep1, swapped) = sr.morph.replica_exchange(rep0, rep1)

In [ ]:
print("Swapped?", swapped)
print(rep0.get_lambda(), rep1.get_lambda())

# Non-equilibrium switching

Direct access to the lambda value of dyamics objects allows it to be changed on-the-fly

In [ ]:
d = mols.dynamics(timestep="4fs", temperature="25oC", lambda_value=0.0, energy_frequency=sr.u("1ps"))

In [ ]:
d.minimise()
d.run("5ps")

In [ ]:
d.get_lambda()

In [ ]:
d.set_lambda(1.0)
d.run("5ps")

In [ ]:
d.get_lambda()

In [ ]:
df = d.energy_trajectory(to_pandas=True)
df